In [2]:
import os
import glob
import datetime
import calendar

from math import ceil

In [3]:
def configuration_file(start_date='19990101', end_date='19990131', walltime=10):
    conf_txt = f"""#!/bin/bash
#PBS -P kl02
#PBS -q normal
#PBS -l walltime={walltime}:00:00
#PBS -l mem=64GB
#PBS -l wd
#PBS -l ncpus=16
#PBS -lstorage=scratch/kl02
source activate radar

python radar_pack.py -s {start_date} -e {end_date} -i /scratch/kl02/vhl548/cpol/v2018/ppi -o /scratch/kl02/vhl548/cpol/v2018/
"""
    return conf_txt

In [19]:
for year in range(1999, 2018):
    for month in range(1, 13):
        if month > 7 and month < 10:
            continue

        if month < 8:
            season_st = ((year - 1) % 100)
            season_nd = ((year) % 100)
        else:
            season_st = (year % 100)
            season_nd = ((year + 1) % 100)
        season = "%02i%02i" % (season_st, season_nd)
 
        indir = "/scratch/kl02/vhl548/cpol/v2018/ppi"
        indir += "/%i/%i%02i" % (year, year, month)
        dirlist = glob.glob(indir + "*")
        flist = glob.glob(indir + "**/*.*")
        
        if len(dirlist) == 0:
            continue
            
        wtime = (len(dirlist) / 3600)
        if wtime > 1:
            wtime = ceil(wtime)
        else:
            wtime = 1

        _, ed = calendar.monthrange(year, month)
        sdatestr = "%i%02i%02i" % (year, month, 1)
        edatestr = "%i%02i%02i" % (year, month, ed)
        f = configuration_file(sdatestr, edatestr, wtime)
        

        fname = "../radar_grids/qgrid_%i%02i.pbs" % (year, month)
        with open(fname, 'w') as fid:
            fid.write(f)